In [4]:
import sys
sys.path.append('/Users/jnimoca/Jose_BI/ashlar/ashlar/')
from ashlar import fileseries, thumbnail,reg
from ashlar.reg import plot_edge_shifts, plot_edge_quality
from ashlar.reg import BioformatsReader
from ashlar.reg import PyramidWriter
from ashlar.viewer import view_edges
# from ashlar.scripts.ashlar import process_axis_flip
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import argparse
import numpy as np
import pandas as pd
# import pims
import napari
import os
from plotly import express as px
import plotly.io as pio
import kaleido
import skimage
import time
from os.path import abspath
def get_datetime():
    return time.strftime("%Y%m%d_%H%M%S")

# Step 1: Get nd2s into readers

In [2]:
def ashlar_fileseries(
        path_to_files:str,
        pattern:str,
        overlap:float,
        width:int,
        height:int,
        layout:str,
        direction:str,
        pixel_size:float,
        cycle_ids: list,
        
        channel_to_align:int, 
        max_shift:int, 
        filter_sigma:int, 
        quiet: bool=False,
        verbose:bool=True,
        output_image_path:str=None
        ):
    

    readers = []
    for i in cycle_ids:
        readers.append(
            fileseries.FileSeriesReader(
                path_to_files,
                pattern=pattern,
                overlap=overlap,
                width=width,
                height=height,
                layout=layout,
                direction=direction,
                pixel_size=pixel_size
            )
        )

    if not quiet:
        print("Stitching and registering input images")
        print('Cycle 0:')
        print('    reading %s' % path_to_files)
    
    aligner = reg.EdgeAligner(reader=readers[0], channel=channel_to_align, max_shift=max_shift, filter_sigma=filter_sigma, verbose=verbose)
    aligner.run()

    df = pd.DataFrame(
        data = {
            "edge_id": [f"edge_{str(i).zfill(4)}" for i in range(0, aligner.all_errors.shape[0])],
            "cycle_id": "cycle_01",
            'X_Error': np.clip(aligner.all_errors, 0, 10),
            'Y_Shift': np.clip([np.linalg.norm(v[0]) for v in aligner._cache.values()], 0.01, np.inf)
        }
    )

    mosaic_args = {}
    mosaic_args['verbose'] = verbose
    mosaic_args['channels'] = list(range(0, readers[0].metadata.num_channels))

    mosaics = []
    mosaics.append(reg.Mosaic(aligner, aligner.mosaic_shape, **mosaic_args))

    layers = []

    for i, reader in enumerate(readers[1:]):

        if not quiet:
            print("Stitching and registering input images")
            print(f'Cycle {i+1}:')
        
        # tmp_reader = BioformatsReader(file)
        tmp_aligner = reg.EdgeAligner(reader=reader, channel=channel_to_align, max_shift=max_shift, filter_sigma=filter_sigma, verbose=verbose)
        tmp_aligner.run()
        tmp_df = pd.DataFrame(
            data = {
                "edge_id": [f"edge_{str(i).zfill(4)}" for i in range(0, tmp_aligner.all_errors.shape[0])],
                "cycle_id": f"cycle_{str(i+2).zfill(2)}",
                'X_Error': np.clip(tmp_aligner.all_errors, 0, 10),
                'Y_Shift': np.clip([np.linalg.norm(v[0]) for v in tmp_aligner._cache.values()], 0.01, np.inf)
            }
        )

        df = pd.concat([df, tmp_df], ignore_index=True)
        
        layer_aligner = reg.LayerAligner(reader, aligner)
        layer_aligner.run()
        layers.append(layer_aligner)
        mosaics.append(reg.Mosaic(layer_aligner, aligner.mosaic_shape))

    print(f"Writing output image to {output_image_path}")
    writer = PyramidWriter(mosaics, output_image_path, verbose=verbose)
    writer.run()

    return df,aligner,layers

In [7]:
df,aligner,layers = ashlar_fileseries(
    path_to_files='../raw/',
    pattern='cycle{i}_core0001_tile{series}.nd2',
    overlap=0.5,
    width=9,
    height=9,
    layout='snake',
    direction='horizontal',
    pixel_size=0.17,
    cycle_ids=['0001','0000'],
    channel_to_align=0,
    max_shift=30,
    filter_sigma=1,
    quiet=False,
    verbose=True,
    output_image_path='../registration/20240408_ashlar_output.ome.tif'
)

Stitching and registering input images
Cycle 0:
    reading ../raw/
    assembling thumbnail 81/81


/opt/homebrew/Caskroom/mambaforge/base/envs/ashlar/lib/python3.10/site-packages/ashlar/reg.py:1367: DataWarning: Some neighboring tiles have zero overlap.
  warnings.warn(message, DataWarning)


    quantifying alignment error 1000/1000
    aligning edge 398/398
Stitching and registering input images
Cycle 1:
    assembling thumbnail 81/81


/opt/homebrew/Caskroom/mambaforge/base/envs/ashlar/lib/python3.10/site-packages/ashlar/reg.py:1367: DataWarning: Some neighboring tiles have zero overlap.
  warnings.warn(message, DataWarning)


    quantifying alignment error 1000/1000
    aligning edge 398/398
    assembling thumbnail 81/81
    estimated cycle offset [y x] = [0. 0.]
Writing output image to ../registration/20240408_ashlar_output.ome.tif
Cycle 0:
    Channel 0:
        merging tile 81/81
    Channel 1:
        merging tile 81/81
    Channel 2:
        merging tile 81/81
    Channel 3:
        merging tile 81/81
Cycle 1:
    Channel 0:
    Channel 1:
    Channel 2:
    Channel 3:
Generating pyramid
    Level 1 (5902 x 5790)
        processing channel 8/8
    Level 2 (2951 x 2895)
        processing channel 8/8
    Level 3 (1476 x 1448)
        processing channel 8/8
    Level 4 (738 x 724)
        processing channel 8/8


In [9]:
df.to_csv('../qc/20240408_ashlar_layers.csv', index=False)

In [13]:
df.head()

,edge_id,cycle_id,X_Error,Y_Shift
0,edge_0000,cycle_01,4.585044,47.900106
1,edge_0001,cycle_01,2.785183,252.131738
2,edge_0002,cycle_01,1.368806,10.903669
3,edge_0003,cycle_01,6.819083,501.999048
4,edge_0004,cycle_01,3.067089,833.400624


In [15]:
df.groupby('cycle_id')

In [20]:
df[df.edge_id == 'edge_0003']

,edge_id,cycle_id,X_Error,Y_Shift
3,edge_0003,cycle_01,6.819083,501.999048
401,edge_0003,cycle_02,6.819083,501.999048


In [10]:
def plot_scatter_shift_error(df:pd.DataFrame):

    fig = px.scatter(df, x='X_Error', y='Y_Shift', color='cycle_id')

    fig.update_layout(
        title='Error vs Shift', 
        height=800,width=1200,
        paper_bgcolor="white", plot_bgcolor="white",
        uniformtext_minsize=12, uniformtext_mode='hide'
    )

    fig.update_yaxes(
        type="log", nticks=7, 
        showline=True, linecolor="black", mirror=True,
        title=dict(
            text='Shift distance (pixels)',
            font=dict(
                family='Arial',
                size=18,
                color='black'
            )   
        )
    )

    fig.update_xaxes(
        showline=True, linecolor="black", mirror=True,
        title=dict(
            text='Error (NCC)', 
            font=dict(
                family='Arial',
                size=18,
                color='black'
            )   
        )
    )

    fig.update_traces(
        marker=dict(
            size=8, 
            opacity=0.6, 
            line=dict(
                width=1, 
                color='DarkSlateGrey'
            )
        )
    )

    return fig

In [14]:
plot_scatter_shift_error(df)

In [21]:
QC_folder = '../qc/test2/'
datetime=get_datetime()

export_path= os.path.join(QC_folder, f"{datetime}_ashlar_QC_plots.pdf")

fig = plot_scatter_shift_error(df)
plt.suptitle(f"Error vs Shift \n -m {aligner.max_shift} --filter-sigma {aligner.filter_sigma}", color='salmon')
pio.write_image(fig, os.path.join(QC_folder, f"{datetime}_Error_vs_Shift_ScatterPlot.pdf"), scale=6, width=600, height=500)

filename = export_path
pdf_pages = PdfPages(filename)

plot_edge_shifts(aligner)
#brighter is more distance
plt.suptitle(f"Edge Shifts : Brighter is more shift \n -m {aligner.max_shift} --filter-sigma {aligner.filter_sigma}", color='salmon')
plt.tight_layout()
plt.gcf().set_size_inches(16, 9)
fig = plt.gcf()
pdf_pages.savefig(fig)
plt.close(fig)

plot_edge_quality(aligner, img=aligner.reader.thumbnail)
plt.suptitle(f"Edge Quality : Brighter is better \n -m {aligner.max_shift} --filter-sigma {aligner.filter_sigma}", color='salmon')
plt.tight_layout()
plt.gcf().set_size_inches(16, 9)
fig = plt.gcf()
pdf_pages.savefig(fig)
plt.close(fig)


for i, layer in enumerate(layers):
    layer.mosaic_shape = aligner.mosaic_shape
    reg.plot_layer_quality(layer, img=aligner.reader.thumbnail)
    plt.gcf().suptitle(f"Cycle {i+1} -m {aligner.max_shift} --filter-sigma {aligner.filter_sigma}", color='salmon')
    plt.gcf().tight_layout()
    plt.gcf().set_size_inches(16, 9)
    fig = plt.gcf()
    pdf_pages.savefig(fig)
    plt.close(fig)

# Close the PdfPages object
pdf_pages.close()

<Figure size 640x480 with 0 Axes>